In [1]:
import sys
import os
import time
import threading
import json
import numpy as np
import cv2
import cv2.aruco as aruco
from scipy.spatial.transform import Rotation as R
import pyzed.sl as sl
import mecademicpy.robot as mdr
from queue import Queue

def get_dh_matrix(a, d, alpha, theta):
    alpha_rad = np.deg2rad(alpha)
    theta_rad = np.deg2rad(theta)
    return np.array([
        [np.cos(theta_rad), -np.sin(theta_rad) * np.cos(alpha_rad),  np.sin(theta_rad) * np.sin(alpha_rad), a * np.cos(theta_rad)],
        [np.sin(theta_rad),  np.cos(theta_rad) * np.cos(alpha_rad), -np.cos(theta_rad) * np.sin(alpha_rad), a * np.sin(theta_rad)],
        [0,                  np.sin(alpha_rad),                     np.cos(alpha_rad),                      d],
        [0,                  0,                                     0,                                      1]
    ], dtype=np.float32)

def forward_kinematics(joint_angles):
    """Compute 3D positions of robot joints using DH parameters."""
    dh_params = [
        {'alpha': -90, 'a': 0,     'd': 0.135, 'theta_offset': 0},
        {'alpha': 0,   'a': 0.135, 'd': 0,     'theta_offset': -90},
        {'alpha': -90, 'a': 0.038, 'd': 0,     'theta_offset': 0},
        {'alpha': 90,  'a': 0,     'd': 0.120, 'theta_offset': 0},
        {'alpha': -90, 'a': 0,     'd': 0,     'theta_offset': 0},
        {'alpha': 0,   'a': 0,     'd': 0.070, 'theta_offset': 0},
        # {'alpha': 0,   'a': 0,     'd': 0.200, 'theta_offset': 0}, # End-effector (Tool)
    ]
    joint_coords = [np.array([0, 0, 0])]
    T = np.eye(4)
    rot_x_180 = R.from_euler('x', 180, degrees=True)
    rot_z_90 = R.from_euler('z', 90, degrees=True)
    T[:3, :3] = (rot_z_90 * rot_x_180).as_matrix()
    base_point = np.array([[0], [0], [0], [1]])
    for i in range(6):
        p = dh_params[i]
        theta = joint_angles[i] + p['theta_offset']
        T_i = get_dh_matrix(p['a'], p['d'], p['alpha'], theta)
        T = T @ T_i
        joint_pos = T @ base_point
        joint_coords.append(joint_pos[:3, 0])
    return np.array(joint_coords, dtype=np.float32)

def project_to_image(joints_3d, rvec, tvec, K, dist):
    """Project 3D joints into 2D image using extrinsics and intrinsics."""
    # Ensure joints_3d is a 2D array
    if joints_3d.ndim == 1:
        joints_3d = joints_3d.reshape(1, -1)
    R_mat, _ = cv2.Rodrigues(rvec)
    pts_cam = (R_mat @ joints_3d.T).T + tvec.reshape(1, 3)
    img_pts, _ = cv2.projectPoints(pts_cam, np.zeros(3), np.zeros(3), K, dist)
    return img_pts.reshape(-1, 2)

In [ ]:
import os
import json
import numpy as np
import cv2
import cv2.aruco as aruco
import pyzed.sl as sl
from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt

# --- 1. 기본 설정 ---
serial = 41182735
calib_path = '/home/zed_box/Documents/VLA/Auto_Needle_Insertion_System/calib/front_41182735_leftcam_calib.json'

marker_size = 0.05  # 5cm
marker_3d_edges = np.array([
    [-marker_size/2,  marker_size/2, 0],
    [ marker_size/2,  marker_size/2, 0],
    [ marker_size/2, -marker_size/2, 0],
    [-marker_size/2, -marker_size/2, 0]
], dtype=np.float32)

# --- 2. ZED 카메라 초기화 ---
zed = sl.Camera()
init = sl.InitParameters()
init.set_from_serial_number(serial)
if zed.open(init) != sl.ERROR_CODE.SUCCESS:
    print(f"[ERROR] Failed to open camera {serial}")
    exit()
runtime = sl.RuntimeParameters()

# --- 3. ArUco 및 캘리브레이션 설정 ---
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_5X5_50)
detector_params = aruco.DetectorParameters()
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01)

with open(calib_path, 'r') as f:
    calib = json.load(f)
K = np.array(calib['camera_matrix'], dtype=np.float32)
dist = np.array(calib['distortion_coeffs'], dtype=np.float32)

# --- 4. 단일 프레임 캡처 및 처리 ---
print("단일 프레임을 캡처합니다...")
if zed.grab(runtime) == sl.ERROR_CODE.SUCCESS:
    img_sl = sl.Mat()
    zed.retrieve_image(img_sl, sl.VIEW.LEFT)
    frame = img_sl.get_data()[:, :, :3].copy()
    
    # 왜곡 보정
    frame_undistorted_bgr = cv2.undistort(frame, K, dist)
    
    # ArUco 검출 및 자세 추정
    gray = cv2.cvtColor(frame_undistorted_bgr, cv2.COLOR_BGR2GRAY)
    corners, ids, _ = aruco.detectMarkers(gray, aruco_dict, parameters=detector_params)

    if ids is not None and len(ids) > 0:
        print(f"✅ {len(ids)}개의 ArUco 마커를 검출했습니다.")
        for i, corner in enumerate(corners):
            marker_id = int(ids[i][0])
            corner = corner.reshape((4, 2))
            cv2.cornerSubPix(gray, corner, winSize=(5, 5), zeroZone=(-1, -1), criteria=criteria)

            ret, rvec, tvec = cv2.solvePnP(marker_3d_edges, corner, K, dist)
            if ret:
                rvec, tvec = cv2.solvePnPRefineLM(marker_3d_edges, corner, K, dist, rvec, tvec)
                # 시각화 (이미지에 직접 그림)
                for pt in corner:
                    cv2.circle(frame_undistorted_bgr, tuple(pt.astype(int)), 4, (255, 0, 0), -1)
                cv2.drawFrameAxes(frame_undistorted_bgr, K, dist, rvec, tvec, marker_size / 2)
                cv2.putText(frame_undistorted_bgr, f"ID: {marker_id}",
                            (int(corner[0][0]), int(corner[0][1]) - 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    else:
        print("🚨 ArUco 마커를 찾지 못했습니다.")
        cv2.putText(frame_undistorted_bgr, "ArUco Not Detected", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)

    # --- 5. Matplotlib으로 최종 결과 이미지 표시 ---
    frame_to_display = cv2.cvtColor(frame_undistorted_bgr, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(12, 8))
    plt.imshow(frame_to_display)
    plt.title("ArUco Detection Result (Single Frame)")
    plt.axis('off')
    plt.show()

else:
    print("[ERROR] 카메라에서 프레임을 가져오는 데 실패했습니다.")

# --- 6. 카메라 종료 ---
zed.close()
print("카메라가 성공적으로 종료되었습니다.")